In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import time
from sklearn.metrics import r2_score 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

import yfinance as yf
from datetime import datetime, timedelta
import os
import math
import pandas_datareader as web
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pandas_datareader import data as web_data
import gradio as gr
s=Service(r"D:\new chrome driver\chromedriver-win64\chromedriver.exe")


In [37]:
def download_chartink_excel(email: str, password: str):
    # Set up Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--no-sandbox")  
    chrome_options.add_argument("--disable-dev-shm-usage")  
    s = Service(r"D:\new chrome driver\chromedriver-win64\chromedriver.exe")


    driver = webdriver.Chrome(service=s, options=chrome_options)

    try:
        driver.get("https://chartink.com/login")
        time.sleep(5)  
        email_inp = driver.find_element(By.ID, 'email')
        email_inp.send_keys(email)
        email_inp.send_keys(Keys.ENTER)

        password_inp = driver.find_element(By.ID, "password")
        password_inp.send_keys(password)

        time.sleep(5)  

        
        driver.get("https://chartink.com/screener/indicatorspr1")
        run_btn = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[5]/div/div/div/div[2]/div/div[2]/div[2]/div[1]/button[1]')
        run_btn.click()
        time.sleep(5)  
        excelbtn = driver.find_element(By.XPATH, '//*[@id="DataTables_Table_0_wrapper"]/div[1]/div/button[3]')
        excelbtn.click()
        time.sleep(1)
        print("FILE DOWNLOADED")
    
    finally:
        
        driver.quit()




In [3]:
df_stock = pd.DataFrame()

In [4]:
def process_stock_data(directory: str):
    global df_stock  
    files = os.listdir(directory)
    csv_files = [file for file in files if file.endswith('.csv')]

    if csv_files:
        most_recent_csv = max(csv_files, key=lambda file: os.path.getmtime(os.path.join(directory, file)))
        most_recent_csv_path = os.path.join(directory, most_recent_csv)

        df1 = pd.read_csv(most_recent_csv_path)

        df1['% Chg'] = df1['% Chg'].str.rstrip('%').astype(float)

        df = df1.sort_values(by='% Chg')

        stock_symbols = df['Symbol'].tolist()
        stock_prices = df['Price'].tolist()
        stock_change = df['% Chg'].tolist()

        stock_to_buy = {
            'Stock': stock_symbols,
            'Price': stock_prices,
            'Change': stock_change
        }

        df_stock = pd.DataFrame(stock_to_buy)
        df_stock['Accuracy'] = np.nan  

        print(df_stock)
    else:
        print("No CSV files found in the directory.")


directory = 'D:/'

In [5]:
def stock_predict(stock, index):
    df = yf.download(f'{stock}.NS', start='2023-09-01', end=end_date.strftime('%Y-%m-%d'))
    if df.empty or df.index.min() > pd.Timestamp('2023-09-01'):
        first_available_date = df.index.min().strftime('%Y-%m-%d') if not df.empty else 'No data'
        df_stock.loc[index, 'Accuracy'] = f"Data starts from {first_available_date}"
        return  
    

    data = df.filter(['Close'])
    dataset = data.values
    training_data_len = math.ceil(len(dataset) * 0.7)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)

    train_data = scaled_data[0:training_data_len, :]
    x_train = []
    y_train = []
    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')


    model.fit(x_train, y_train, batch_size=16, epochs=20)

    test_data = scaled_data[training_data_len - 60:, :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    valid = data[training_data_len:]
    valid.loc[:, 'Predictions'] = predictions
    r2 = r2_score(valid['Close'], valid['Predictions'])

    df_stock.loc[index, 'Accuracy'] = r2

end_date = datetime.now() - timedelta(days=1)



In [6]:
def filter_stocks_by_accuracy():
    global df_stock
    df_stock['Accuracy'] = pd.to_numeric(df_stock['Accuracy'], errors='coerce')
    df_stock = df_stock[df_stock['Accuracy'] > 0.4]
    

In [7]:
def calculate_overall_accuracy():
    return df_stock['Accuracy'].mean()



In [19]:
# def allocate_funds(df_stock, purse_size, num_stocks, chg_weight=0.5, acc_weight=0.5):
    
#     df_stock['Change'] = pd.to_numeric(df_stock['Change'], errors='coerce')
#     df_stock['Accuracy'] = pd.to_numeric(df_stock['Accuracy'], errors='coerce')
#     df_stock.dropna(subset=['Change', 'Accuracy'], inplace=True)

#     df_stock['%chg_norm'] = (df_stock['Change'] - df_stock['Change'].min()) / (df_stock['Change'].max() - df_stock['Change'].min())
#     df_stock['Accuracy_norm'] = (df_stock['Accuracy'] - df_stock['Accuracy'].min()) / (df_stock['Accuracy'].max() - df_stock['Accuracy'].min())
    
#     df_stock['Score'] = (chg_weight * df_stock['%chg_norm']) + (acc_weight * df_stock['Accuracy_norm'])
    
#     df_top_stocks = df_stock.sort_values(by='Score', ascending=False).head(num_stocks)
    
#     total_score = df_top_stocks['Score'].sum()
    
#     df_top_stocks['Allocation'] = (df_top_stocks['Score'] / total_score) * purse_size
    
#     df_top_stocks['Quantity'] = np.floor(df_top_stocks['Allocation'] / df_top_stocks['Price']).astype(int)

#     print(f"\nOptimal allocation of funds from purse size ₹{purse_size}:")
#     for i, row in df_top_stocks.iterrows():
#         print(f"Stock: {row['Stock']}, Allocated Funds: ₹{row['Allocation']:.2f}, Quantity to Purchase: {int(row['Quantity'])}")
    
#     return df_top_stocks[['Stock', 'Price', 'Change', 'Accuracy', 'Allocation', 'Quantity']]

In [47]:

def allocate_funds(df_stock, purse_size, num_stocks, chg_weight=0.5, acc_weight=0.5):
    
    df_stock['Change'] = pd.to_numeric(df_stock['Change'], errors='coerce')
    df_stock['Accuracy'] = pd.to_numeric(df_stock['Accuracy'], errors='coerce')
    df_stock.dropna(subset=['Change', 'Accuracy'], inplace=True)

    df_stock['%chg_norm'] = (df_stock['Change'] - df_stock['Change'].min()) / (df_stock['Change'].max() - df_stock['Change'].min())
    df_stock['Accuracy_norm'] = (df_stock['Accuracy'] - df_stock['Accuracy'].min()) / (df_stock['Accuracy'].max() - df_stock['Accuracy'].min())
    
    df_stock['Score'] = (chg_weight * df_stock['%chg_norm']) + (acc_weight * df_stock['Accuracy_norm'])
    
    df_top_stocks = df_stock.sort_values(by='Score', ascending=False).head(num_stocks)
    
    total_score = df_top_stocks['Score'].sum()
    
    df_top_stocks['Allocation'] = (df_top_stocks['Score'] / total_score) * purse_size
    
    df_top_stocks['Quantity'] = np.floor(df_top_stocks['Allocation'] / df_top_stocks['Price']).astype(int)

    print(f"\nOptimal allocation of funds from purse size ₹{purse_size}:")
    for i, row in df_top_stocks.iterrows():
        print(f"Stock: {row['Stock']}, Allocated Funds: ₹{row['Allocation']:.2f}, Quantity to Purchase: {int(row['Quantity'])}")
    
    return df_top_stocks[['Stock', 'Price', 'Change', 'Accuracy', 'Allocation', 'Quantity']]


In [48]:
#isko run krna
global df_stock
directory = "D:/"
download_chartink_excel("dipika12jain@gmail.com", "Dipika@123") 
process_stock_data(directory)
    
for index, row in df_stock.iterrows():
    stock_predict(row['Stock'], index)  
    
filter_stocks_by_accuracy()
def run_investment_strategy(purse_size, num_stocks):
    df_allocation = allocate_funds(df_stock, purse_size, num_stocks)
    return df_allocation

with gr.Blocks() as demo:
    gr.Markdown("## Stock Investment Strategy")
    
    purse_size_input = gr.Number(label="Total Purse Size (in ₹)", value=10000)
    num_stocks_input = gr.Number(label="Number of Stocks to Invest In", value=5)
    
    output = gr.Dataframe(label="Investment Allocation Details", interactive=False)
    
    submit_button = gr.Button("Run Investment Strategy")
    
    def on_submit(purse_size, num_stocks): 
        allocation_result = run_investment_strategy(purse_size, num_stocks)
        return allocation_result
    
    submit_button.click(on_submit, inputs=[purse_size_input, num_stocks_input], outputs=output)


demo.launch()


FILE DOWNLOADED


[*********************100%***********************]  1 of 1 completed

        Stock     Price  Change  Accuracy
0  ARTEMISMED    280.05    3.86       NaN
1    AEGISLOG    805.05    4.30       NaN
2      NBIFIN   2464.20    4.83       NaN
3   POLICYBZR   1724.50    5.61       NaN
4  NEULANDLAB  15590.25    6.30       NaN


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - loss: 0.0461
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - loss: 0.0052
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0028
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - loss: 0.0027
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 0.0023
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0017
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 0.0021
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.0018
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 0.0017
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.0018
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.0019
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0019
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0019
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - loss: 0.0023
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - loss: 0.00

C:\Users\hp\AppData\Local\Temp\ipykernel_16860\3350366583.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid.loc[:, 'Predictions'] = predictions
[*********************100%***********************]  1 of 1 completed
C:\Users\hp\AppData\Local\Temp\ipykernel_16860\3350366583.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Data starts from 2024-05-30' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_stock.loc[index, 'Accuracy'] = f"Data starts from {first_available_date}"
[*********************100%***********************]  1 of 1 completed


...

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - loss: 0.0881
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0097
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0084
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.0039
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0024
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 0.0020
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0018
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0017
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0018
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0019
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.0017
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0024
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0017
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.0013
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 

C:\Users\hp\AppData\Local\Temp\ipykernel_16860\3350366583.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid.loc[:, 'Predictions'] = predictions
[*********************100%***********************]  1 of 1 completed


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 70ms/step - loss: 0.0382
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.0055
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.0035
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0023
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 0.0021
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0018
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0017
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0016
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0015
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.0015
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.0016
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0013
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.0015
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - loss: 0.0013
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.00

C:\Users\hp\AppData\Local\Temp\ipykernel_16860\3350366583.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid.loc[:, 'Predictions'] = predictions


* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
